## Importando Bibliotecas

In [ ]:
#!pip install -r requirements

In [1]:
import yfinance as yf
import pandas as pd

In [7]:
# Configurar para exibir mais colunas e linhas no terminal
pd.set_option('display.max_rows', None)  # Exibir todas as linhas
pd.set_option('display.max_columns', None)  # Exibir todas as colunas
pd.set_option('display.width', None)  # Ajusta a largura para não quebrar
pd.set_option('display.expand_frame_repr', False)  # Evita a quebra de colunas

## Coleta dos Dados

In [ ]:
# Especificar a ação
symbol = 'NVDA' #ou outra empresa de sua escolha
start_date = '2014-01-01'
end_date = '2024-12-20'

# Baixar os dados
df = yf.download(symbol, start=start_date, end=end_date)

# Visualizar os primeiros dados
print(df.head())


## Desenvolvimento do Modelo LSTM

### Pré-processamento dos Dados
Precisamos normalizar os dados e preparar a série temporal para ser utilizada pelo modelo LSTM.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Selecionar apenas a coluna de "Close" (valor de fechamento)
data = df[['Close']]

# Normalizar os dados entre 0 e 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Dividir os dados em treino e teste (80% para treino e 20% para teste)
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Função para criar o dataset em janelas temporais
def create_dataset(data, window_size):
    x = []
    y = []
    for i in range(window_size, len(data)):
        x.append(data[i-window_size:i, 0])
        y.append(data[i, 0])
    return np.array(x), np.array(y)

# Definir o tamanho da janela temporal
window_size = 60
x_train, y_train = create_dataset(train_data, window_size)
x_test, y_test = create_dataset(test_data, window_size)

# Reshape para [samples, time steps, features] (necessário para o LSTM)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1], 1)


### Construção e Treinamento do Modelo LSTM
Aqui, vamos criar o modelo de rede neural utilizando Keras com LSTM.


In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

# Criar o modelo
model = Sequential()

# Adicionar camadas LSTM
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=False))
model.add(Dropout(0.2))

# Camada de saída
model.add(Dense(units=1))

# Compilar o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

# Treinar o modelo
model.fit(x_train, y_train, epochs=10, batch_size=32)


## Avaliação do Modelo
Depois de treinar, podemos avaliar o modelo nos dados de teste.

In [ ]:
# Fazer as previsões
predictions = model.predict(x_test)

# Inverter a normalização para trazer os dados de volta à escala original
predictions = scaler.inverse_transform(predictions)
y_test_unscaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# Avaliar com RMSE
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test_unscaled, predictions))
print(f'RMSE: {rmse}')

## Salvamento e Exportação do Modelo
Agora, salvamos o modelo treinado.

In [ ]:
model.save('lstm_stock_model.h5')

## Deploy do Modelo em uma API
Vamos criar uma API utilizando Flask para servir o modelo.

In [ ]:
from flask import Flask, request, jsonify
import numpy as np
from keras.models import load_model

# Carregar o modelo treinado
model = load_model('lstm_stock_model.h5')

app = Flask(__name__)

# Definir a rota para fazer previsões
@app.route('/predict', methods=['POST'])
def predict():
    # Receber os dados históricos do cliente
    data = request.json['data']
    data = np.array(data).reshape(-1, 1)
    
    # Normalizar os dados recebidos
    scaled_data = scaler.transform(data)
    
    # Preparar os dados para o modelo (usando uma janela de 60 dias)
    x_input = scaled_data[-60:].reshape(1, 60, 1)
    
    # Fazer a previsão
    prediction = model.predict(x_input)
    
    # Inverter a normalização da previsão
    predicted_price = scaler.inverse_transform(prediction)
    
    return jsonify({'predicted_price': predicted_price[0][0]})

if __name__ == '__main__':
    app.run(debug=True)
